## Import thư viện

In [8]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

## Đọc dữ liệu từ bộ dataset

In [2]:
df = pd.read_csv(filepath_or_buffer='./analysis_df.csv', delimiter=',')

## Tiền xử lý dữ liệu để huấn luyện

### Drop NaN value

In [ ]:
model_df = df[df['Salary'].notna()]

### Feature Engineering

In [ ]:
salary_ranges = pd.qcut(model_df['Salary'], q=3, labels=['Low', 'Medium', 'High'])

model_df['Salary Range'] = salary_ranges

features = ['Age', 'Title', 'Formal Education', 'Coding Experience', 'ML Experience', 'Country']
target = 'Salary Range'

### Categorical Label Encoding

In [ ]:
label_encoder = LabelEncoder()
for feature in features:
    if model_df[feature].dtype == 'object':
        model_df[feature] = label_encoder.fit_transform(model_df[feature])

## Chia tập dữ liệu thành train và set

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(model_df['features'], model_df[target], test_size=0.2, random_state=42)

## Huấn luyện mô hình bằng KNN (K-Nearest Neighbor) Classification

### Bài toán: Phân lớp khoảng lương của 1 người tham gia trả lời dựa vào 1 số thông tin cá nhân của họ

**Các cột dữ liệu được sử dụng làm feature:**
- Age: tuổi
- Title: vị trí/vai trò
- Formal Education: bằng cấp
- Coding Experience: số năm kinh nghiệm lập trình
- ML Experience: số năm kinh nghiệm về machine learning
- Country: quốc gia

**Cột dữ liệu làm label:**
- Salary Range: khoảng lương của người đó (Low-Medium-High)

**Mô hình:** K-Nearest Neighbor

**Tối ưu hoá tham số**: Grid Search

**Metrics đánh giá:** Accuracy

In [7]:
knn_classifier = KNeighborsClassifier()

param_grid = {'n_neighbors': list(range(10, 30, 2))}

grid_search = GridSearchCV(knn_classifier, param_grid, cv=len(param_grid['n_neighbors']))
grid_search.fit(X_train, y_train)

best_n_neighbors = grid_search.best_params_['n_neighbors']
best_classifier = grid_search.best_estimator_

predictions = best_classifier.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
print("Best n_neighbors:", best_n_neighbors)
print("Accuracy:", accuracy)

C:\Users\Admin\AppData\Local\Temp\ipykernel_14252\957932630.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df['Salary Range'] = salary_ranges


Best n_neighbors: 26
Accuracy: 0.6584500098599881
